In [ ]:
conda install opencv

In [ ]:
conda install -c conda-forge dlib

In [1]:
# Import required packages
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

## 1. Load the datasets

For the project, we provide a training set with 50000 images in the directory `../data/images/` with:
- noisy labels for all images provided in `../data/noisy_label.csv`;
- clean labels for the first 10000 images provided in `../data/clean_labels.csv`. 

In [2]:
# [DO NOT MODIFY THIS CELL]

# load the images
n_img = 50000
n_noisy = 40000
n_clean_noisy = n_img - n_noisy
imgs = np.empty((n_img,32,32,3))
for i in range(n_img):
    img_fn = f'../data/images/{i+1:05d}.png'
    imgs[i,:,:,:]=cv2.cvtColor(cv2.imread(img_fn),cv2.COLOR_BGR2RGB)

# load the labels
clean_labels = np.genfromtxt('../data/clean_labels.csv', delimiter=',', dtype="int8")
noisy_labels = np.genfromtxt('../data/noisy_labels.csv', delimiter=',', dtype="int8")

For illustration, we present a small subset (of size 8) of the images with their clean and noisy labels in `clean_noisy_trainset`. You are encouraged to explore more characteristics of the label noises on the whole dataset. 

In [3]:
# [DO NOT MODIFY THIS CELL]

fig = plt.figure()

ax1 = fig.add_subplot(2,4,1)
ax1.imshow(imgs[0]/255)
ax2 = fig.add_subplot(2,4,2)
ax2.imshow(imgs[1]/255)
ax3 = fig.add_subplot(2,4,3)
ax3.imshow(imgs[2]/255)
ax4 = fig.add_subplot(2,4,4)
ax4.imshow(imgs[3]/255)
ax1 = fig.add_subplot(2,4,5)
ax1.imshow(imgs[4]/255)
ax2 = fig.add_subplot(2,4,6)
ax2.imshow(imgs[5]/255)
ax3 = fig.add_subplot(2,4,7)
ax3.imshow(imgs[6]/255)
ax4 = fig.add_subplot(2,4,8)
ax4.imshow(imgs[7]/255)

# The class-label correspondence
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# print clean labels
print('Clean labels:')
print(' '.join('%5s' % classes[clean_labels[j]] for j in range(8)))
# print noisy labels
print('Noisy labels:')
print(' '.join('%5s' % classes[noisy_labels[j]] for j in range(8)))


Clean labels:
 frog truck truck  deer   car   car  bird horse
Noisy labels:
  cat   dog truck  frog   dog  ship  bird  deer


## 2. The predictive model

We consider a baseline model directly on the noisy dataset without any label corrections. RGB histogram features are extracted to fit a logistic regression model.

### 2.1. Baseline Model

In [ ]:
# [DO NOT MODIFY THIS CELL]
# RGB histogram dataset construction
no_bins = 6
bins = np.linspace(0,255,no_bins) # the range of the rgb histogram
target_vec = np.empty(n_img)
feature_mtx = np.empty((n_img,3*(len(bins)-1)))
i = 0
for i in range(n_img):
    # The target vector consists of noisy labels
    target_vec[i] = noisy_labels[i]
    
    # Use the numbers of pixels in each bin for all three channels as the features
    feature1 = np.histogram(imgs[i][:,:,0],bins=bins)[0] 
    feature2 = np.histogram(imgs[i][:,:,1],bins=bins)[0]
    feature3 = np.histogram(imgs[i][:,:,2],bins=bins)[0]
    
    # Concatenate three features
    feature_mtx[i,] = np.concatenate((feature1, feature2, feature3), axis=None)
    i += 1

In [ ]:
# [DO NOT MODIFY THIS CELL]
# Train a logistic regression model 
clf = LogisticRegression(random_state=0).fit(feature_mtx, target_vec)

For the convenience of evaluation, we write the following function `predictive_model` that does the label prediction. **For your predictive model, feel free to modify the function, but make sure the function takes an RGB image of numpy.array format with dimension $32\times32\times3$  as input, and returns one single label as output.**

In [ ]:
# [DO NOT MODIFY THIS CELL]
def baseline_model(image):
    '''
    This is the baseline predictive model that takes in the image and returns a label prediction
    '''
    feature1 = np.histogram(image[:,:,0],bins=bins)[0]
    feature2 = np.histogram(image[:,:,1],bins=bins)[0]
    feature3 = np.histogram(image[:,:,2],bins=bins)[0]
    feature = np.concatenate((feature1, feature2, feature3), axis=None).reshape(1,-1)
    return clf.predict(feature)

### 2.2. Model I

In [4]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical 

In [5]:
# the labels used for training and validation
model1_label = np.concatenate((clean_labels, noisy_labels[10000:]), axis=None)
X_train, X_test, y_train, y_test = train_test_split(imgs, model1_label, test_size=10000, random_state=5243)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40000, 32, 32, 3)
(10000, 32, 32, 3)
(40000,)
(10000,)


In [7]:
print(y_train)

[1 6 1 ... 4 7 1]


In [8]:
print(np.eye(10)[y_train].shape)

(40000, 10)


In [9]:
# one-hot encoded vectors

y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]


In [10]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


In [11]:
# Normalization
X_train /= 255
X_test /= 255

In [12]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40000, 32, 32, 3)
(10000, 32, 32, 3)
(40000, 10)
(10000, 10)


In [ ]:
import time

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Nadam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

# try different optimizers
opt = Adam(learning_rate=0.01)
lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.90)
sgd = SGD(learning_rate=lr_schedule)

opt2 = RMSprop(
    learning_rate=lr_schedule,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-07,
    centered=False,
    name="RMSprop",
)

opt3= Nadam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
history = model.fit(X_train, y_train, epochs=15, batch_size=64, verbose=2, validation_split=0.2)

Epoch 1/15
500/500 - 44s - loss: 2.2713 - accuracy: 0.1510 - val_loss: 2.2400 - val_accuracy: 0.1949
Epoch 2/15
500/500 - 47s - loss: 2.2353 - accuracy: 0.1896 - val_loss: 2.2251 - val_accuracy: 0.2090
Epoch 3/15
500/500 - 43s - loss: 2.2152 - accuracy: 0.2098 - val_loss: 2.2142 - val_accuracy: 0.2130
Epoch 4/15
500/500 - 44s - loss: 2.1973 - accuracy: 0.2234 - val_loss: 2.1934 - val_accuracy: 0.2269
Epoch 5/15
500/500 - 44s - loss: 2.1818 - accuracy: 0.2340 - val_loss: 2.1885 - val_accuracy: 0.2336
Epoch 6/15
500/500 - 43s - loss: 2.1678 - accuracy: 0.2462 - val_loss: 2.1754 - val_accuracy: 0.2419
Epoch 7/15
500/500 - 43s - loss: 2.1551 - accuracy: 0.2542 - val_loss: 2.1692 - val_accuracy: 0.2463
Epoch 8/15
500/500 - 43s - loss: 2.1447 - accuracy: 0.2619 - val_loss: 2.1615 - val_accuracy: 0.2509
Epoch 9/15
500/500 - 45s - loss: 2.1342 - accuracy: 0.2692 - val_loss: 2.1603 - val_accuracy: 0.2630
Epoch 10/15
500/500 - 46s - loss: 2.1225 - accuracy: 0.2756 - val_loss: 2.1479 - val_accura

In [15]:
score = model.evaluate(X_test, y_test, batch_size=128, verbose=0)

In [16]:
print(model.metrics_names)
print(score)

['loss', 'accuracy']
[2.1339752674102783, 0.27549999952316284]


In [18]:
import time
start = time.clock()
score = model.evaluate(X_test, y_test, batch_size=128, verbose=0)
end = time.clock()
print(end-start)

/anaconda3/envs/envTF22/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


9.503897000000052


/anaconda3/envs/envTF22/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.


In [19]:
from tensorflow.keras.layers import Dropout
model2 = Sequential()

model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(256, activation='relu'))

model2.add(Dropout(0.5))
model2.add(Dense(10, activation='softmax'))

In [20]:
model2.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
start = time.perf_counter()
history = model2.fit(X_train, y_train, batch_size=32, epochs=15, verbose=2, validation_split=0.2)
end = time.perf_counter()

/anaconda3/envs/envTF22/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Epoch 1/15
1000/1000 - 85s - loss: 2.2777 - accuracy: 0.1426 - val_loss: 2.2404 - val_accuracy: 0.1824
Epoch 2/15
1000/1000 - 82s - loss: 2.2419 - accuracy: 0.1791 - val_loss: 2.2219 - val_accuracy: 0.1951
Epoch 3/15
1000/1000 - 79s - loss: 2.2239 - accuracy: 0.1998 - val_loss: 2.2059 - val_accuracy: 0.2150
Epoch 4/15
1000/1000 - 80s - loss: 2.2049 - accuracy: 0.2096 - val_loss: 2.1848 - val_accuracy: 0.2329
Epoch 5/15
1000/1000 - 79s - loss: 2.1943 - accuracy: 0.2218 - val_loss: 2.1752 - val_accuracy: 0.2409
Epoch 6/15
1000/1000 - 79s - loss: 2.1805 - accuracy: 0.2314 - val_loss: 2.1728 - val_accuracy: 0.2391
Epoch 7/15
1000/1000 - 79s - loss: 2.1710 - accuracy: 0.2390 - val_loss: 2.1633 - val_accuracy: 0.2534
Epoch 8/15
1000/1000 - 82s - loss: 2.1616 - accuracy: 0.2487 - val_loss: 2.1531 - val_accuracy: 0.2589
Epoch 9/15
1000/1000 - 84s - loss: 2.1530 - accuracy: 0.2540 - val_loss: 2.1502 - val_accuracy: 0.2605
Epoch 10/15
1000/1000 - 80s - loss: 2.1453 - accuracy: 0.2587 - val_loss:

/anaconda3/envs/envTF22/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
print(end-start)

3453.9307980000003


In [23]:
print(start)

2302.545481


In [24]:
print(end)

5756.476279


In [ ]:
# [BUILD A MORE SOPHISTICATED PREDICTIVE MODEL]

# write your code here...

def model_I(image):
    '''
    This function should takes in the image of dimension 32*32*3 as input and returns a label prediction
    '''
    # write your code here...
    
    
    
    return model

### 2.3. Model II

In [ ]:
# [ADD WEAKLY SUPERVISED LEARNING FEATURE TO MODEL I]

# write your code here...

def model_II(image):
    '''
    This function should takes in the image of dimension 32*32*3 as input and returns a label prediction
    '''
    # write your code here...

## 3. Evaluation

For assessment, we will evaluate your final model on a hidden test dataset with clean labels by the `evaluation` function defined as follows. Although you will not have the access to the test set, the function would be useful for the model developments. For example, you can split the small training set, using one portion for weakly supervised learning and the other for validation purpose. 

In [ ]:
# [DO NOT MODIFY THIS CELL]
def evaluation(model, test_labels, test_imgs):
    y_true = test_labels
    y_pred = []
    for image in test_imgs:
        y_pred.append(model(image))
    print(classification_report(y_true, y_pred))

In [ ]:
# [DO NOT MODIFY THIS CELL]
# This is the code for evaluating the prediction performance on a testset
# You will get an error if running this cell, as you do not have the testset
# Nonetheless, you can create your own validation set to run the evlauation
n_test = 10000
test_labels = np.genfromtxt('../data/test_labels.csv', delimiter=',', dtype="int8")
test_imgs = np.empty((n_test,32,32,3))
for i in range(n_test):
    img_fn = f'../data/test_images/test{i+1:05d}.png'
    test_imgs[i,:,:,:]=cv2.cvtColor(cv2.imread(img_fn),cv2.COLOR_BGR2RGB)
evaluation(baseline_model, test_labels, test_imgs)

The overall accuracy is $0.24$, which is better than random guess (which should have a accuracy around $0.10$). For the project, you should try to improve the performance by the following strategies:

- Consider a better choice of model architectures, hyperparameters, or training scheme for the predictive model;
- Use both `clean_noisy_trainset` and `noisy_trainset` for model training via **weakly supervised learning** methods. One possible solution is to train a "label-correction" model using the former, correct the labels in the latter, and train the final predictive model using the corrected dataset.
- Apply techniques such as $k$-fold cross validation to avoid overfitting;
- Any other reasonable strategies.